In [1]:
import hgtk
import numpy as np

import nengo
import tensorflow as tf

import nengo_dl

train_data = open('ratings_train_small.txt', 'r', encoding='utf-8')
test_data = open('ratings_test_small.txt', 'r', encoding='utf-8')

train_lines = train_data.readlines()
test_lines = test_data.readlines()

In [2]:
del train_lines[0]
del test_lines[0]
train_lines[0]

'9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0\n'

In [3]:
total_cho = hgtk.josa.CHO
total_joong = hgtk.josa.JOONG
total_jong = hgtk.josa.JONG
print(total_cho)
print(total_jong)
print(total_joong)
print('total_cho len: ' + str(len(total_cho)))
print('total_joong len: ' + str(len(total_joong)))
print('total_jong len: ' + str(len(total_jong)))
print('total sent len = ' + str(len(total_cho) + len(total_joong) + len(total_jong)))

('ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ')
('', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ')
('ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ')
total_cho len: 19
total_joong len: 21
total_jong len: 28
total sent len = 68


In [4]:
max_sent_size = 68 * 68
print(max_sent_size)

4624


In [5]:
def get_sent_vec(ori_sent):
    sub_num = 68 - len(ori_sent)
    if sub_num > 0:
        for num in range(0, sub_num):
            ori_sent += " "
    if len(ori_sent) > 68:
        ori_sent = ori_sent[0:68]
    jamo_sent = hgtk.text.decompose(ori_sent)
    jamo_sent = jamo_sent.replace('ᴥ', '')
    #print(jamo_sent + '\n')

    many_hot = np.zeros(len(total_cho + total_joong + total_jong))
    sent_hot = np.zeros((len(ori_sent),len(many_hot)))
    word_idx = 0

    for word in ori_sent:
        #print("input_word: " + str(word))
        if(hgtk.checker.is_hangul(word)):
            split_word = hgtk.letter.decompose(word)
            cho_idx = 0
            joong_idx = 0
            jong_idx = 0
            one_hot_cho = np.zeros(len(hgtk.josa.CHO))
            one_hot_joong = np.zeros(len(hgtk.josa.JOONG))
            one_hot_jong = np.zeros(len(hgtk.josa.JONG))
            if(len(split_word) > 2):
                for cho in total_cho:
                    if(cho == split_word[0]):
                        one_hot_cho[cho_idx] = 1
                        break
                    else:
                        cho_idx+= 1
                for joong in total_joong:
                    if(joong == split_word[1]):
                        one_hot_joong[joong_idx] = 1
                        break
                    else:
                        joong_idx+= 1
                for jong in total_jong:
                    if(jong == split_word[2]):
                        one_hot_jong[jong_idx] = 1
                        break
                    else:
                        jong_idx+= 1
            else:
                for cho in total_cho:
                    if(cho == split_word[0]):
                        one_hot_cho[cho_idx] = 1
                        break
                    else:
                        cho_idx+= 1
                for joong in total_joong:
                    if(joong == split_word[1]):
                        one_hot_joong[joong_idx] = 1
                        break
                    else:
                        joong_idx+= 1
            many_hot = np.append(one_hot_cho, one_hot_joong)
            many_hot = np.append(many_hot, one_hot_jong)

        else:
            many_hot = np.zeros(len(total_cho + total_joong + total_jong))
        sent_hot[word_idx] = many_hot
        word_idx += 1
    return sent_hot.reshape(1, max_sent_size)[0]

In [6]:
sent_hot = get_sent_vec(train_lines[1].split('\t')[1])
print(sent_hot)
print(len(sent_hot))

[0. 0. 0. ... 0. 0. 0.]
4624


In [7]:
train_data_array = []
sent_vec_array = []
label_array = []
for line in train_lines:
    sent = line.split('\t')[1]
    label = line.split('\t')[2].replace('\n', '')
    sent_vec = get_sent_vec(sent)
    sent_vec_array.append(list(sent_vec))
    label_array.append(int(label))

train_data_array = [np.array(sent_vec_array), np.array(label_array)]

In [8]:
test_data_array

[array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]), array([1, 0, 0, ..., 0, 1, 0])]

In [7]:
test_data_array = []
sent_vec_array = []
label_array = []
for line in test_lines:
    sent = line.split('\t')[1]
    label = line.split('\t')[2].replace('\n', '')
    sent_vec = get_sent_vec(sent)
    sent_vec_array.append(list(sent_vec))
    label_array.append(int(label))

test_data_array = [np.array(sent_vec_array), np.array(label_array)]

In [9]:
len(train_data_array[0][0])

4624

In [31]:
(68*68) / 68*3


204.0

In [56]:
with nengo.Network() as net:
    # set some default parameters for the neurons that will make
    # the training progress more smoothly
    net.config[nengo.Ensemble].max_rates = nengo.dists.Choice([100])
    net.config[nengo.Ensemble].intercepts = nengo.dists.Choice([0])
    neuron_type = nengo.LIF(amplitude=0.01)
    #neuron_type = nengo.SpikingRectifiedLinear()
    
    # we'll make all the nengo objects in the network
    # non-trainable. we could train them if we wanted, but they don't
    # add any representational power. note that this doesn't affect 
    # the internal components of tensornodes, which will always be 
    # trainable or non-trainable depending on the code written in 
    # the tensornode.
    nengo_dl.configure_settings(trainable=False)

    # the input node that will be used to feed in input images 
    inp = nengo.Node([0] * 4624)

    # add the first convolutional layer
    x = nengo_dl.tensor_layer(
        inp, tf.layers.conv1d, shape_in=(68*68, 1), filters=1, kernel_size=68*3)

    # apply the neural nonlinearity
    x = nengo_dl.tensor_layer(x, neuron_type)
# filters랑 shape_in의 마지막 숫자랑 맞춰줘야해 
# kernel size만큼 돌면 남는게 output임
    # add another convolutional layer
    x = nengo_dl.tensor_layer(
        x, tf.layers.conv1d, shape_in=(68*68-(68*3-1), 1), filters=1, kernel_size=68*2)
    x = nengo_dl.tensor_layer(x, neuron_type)

    # add a pooling layer
    x = nengo_dl.tensor_layer(
        x, tf.layers.average_pooling1d, shape_in=(68*68-(68*3-1)-(68*2-1), 1), pool_size=2, strides=2)

    # another convolutional layer
    x = nengo_dl.tensor_layer(
        x, tf.layers.conv1d, shape_in=(2143, 1), filters=1, kernel_size=68*2)
    x = nengo_dl.tensor_layer(x, neuron_type)

    # another pooling layer
    x = nengo_dl.tensor_layer(
        x, tf.layers.average_pooling1d, shape_in=(2143-(68*2-1), 1), pool_size=2, strides=2)


    # linear readout
    x = nengo_dl.tensor_layer(x, tf.layers.dense, units=2)

    # we'll create two different output probes, one with a filter
    # (for when we're simulating the network over time and
    # accumulating spikes), and one without (for when we're
    # training the network using a rate-based approximation)
    out_p = nengo.Probe(x)
    out_p_filt = nengo.Probe(x, synapse=0.1)

In [57]:


minibatch_size = 16
sim = nengo_dl.Simulator(net, minibatch_size=minibatch_size)



Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:01                                               #########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

In [58]:
data = train_data_array

In [59]:
for data in (train_data_array, test_data_array):
    one_hot = np.zeros((data[0].shape[0], 2))
    one_hot[np.arange(data[0].shape[0]), data[1]] = 1
    data[1] = one_hot

In [60]:
# add the single timestep to the training data
train_data_array = {inp: train_data_array[0][:, None, :],
              out_p: train_data_array[1][:, None, :]}

# when testing our network with spiking neurons we will need to run it 
# over time, so we repeat the input/target data for a number of 
# timesteps. we're also going to reduce the number of test images, just 
# to speed up this example.
n_steps = 30
test_data_array = {
    inp: np.tile(test_data_array[0][:minibatch_size*2, None, :],
                 (1, n_steps, 1)),
    out_p_filt: np.tile(test_data_array[1][:minibatch_size*2, None, :],
                        (1, n_steps, 1))}



In [61]:
def objective(outputs, targets): 
    return tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=outputs, labels=targets)

In [62]:
opt = tf.train.RMSPropOptimizer(learning_rate=0.001)

In [63]:
def classification_error(outputs, targets):
    return 100 * tf.reduce_mean(
        tf.cast(tf.not_equal(tf.argmax(outputs[:, -1], axis=-1), 
                             tf.argmax(targets[:, -1], axis=-1)),
                tf.float32))


print("error before training: %.2f%%" % sim.loss(
    test_data_array, {out_p_filt: classification_error}))

Calculation finished in 0:00:01                                                
error before training: 53.12%


In [64]:
do_training = True
if do_training:
    # run training
    sim.train(train_data_array, opt, objective={out_p: objective}, n_epochs=10)
    
    # save the parameters to file
    sim.save_params("model_output/nsmc_params")





|                   Training (0%)                  | ETA:  --:--:-- (loss: ---)

/home/banana/.local/lib/python3.5/site-packages/nengo_dl/utils.py:489: UserWarning: Number of data elements (9999) is not an even multiple of minibatch size (16); inputs will be truncated
  (n_inputs, minibatch_size)))


Training finished in 0:01:57 (loss: 11.0789)                                   


In [67]:
print("error after training: %.2f%%" % sim.loss(
    test_data_array, {out_p_filt: classification_error}))

Calculation finished in 0:00:00                                                
error after training: 53.12%
